# A111电压源基本操作以及相关测试

A111核心板上共有5组电压源，负责为A111芯片提供操作电压：

- VOUTA（N_LVDAC）
- VOUTB（P_LVDAC）
- VOUTC（SA）
- VOUTD（N_HVDAC）
- VOUTE（P_HVDAC）

FPGA上的CPU通过SPI总线(spi1 & spi2)分别连接ad5686r和ad5761r，从而控制各路电压；

其中VOUTA~VOUTD由ad5686r控制，VOUTE由ad5761r控制：
<div><br /><a href="pic/vsrc1.png"><img style="float: left;" src="pic/vsrc1.png" width=900 title="点击放大看原图"></a><br /></div>

设置好电压源的电压后，可以通过测量以下触点来验证设置是否成功：
<div><br /><a href="pic/vsrc2.jpg"><img style="float: left;" src="pic/vsrc2.jpg" width=500 title="点击放大看原图"></a></div>

## 基本操作

### 电压源设置（电压单位：mV）

In [ ]:
import time
import ipywidgets as widgets
import a111sdk

l = ["VOUTA", "VOUTB", "VOUTC", "VOUTD", "VOUTE"]
chk = [widgets.Checkbox(description=a) for a in l]

va = 552 
vb = 958 
vc = 200 
vd = 1000
ve = 5000

# 打开设备（根据驱动的设计，设备打开后会自动上电，因此不需要主动调用“a111_power_on()”）
a111sdk.open_a111()

a111sdk.a111_vsource_set(a111sdk.VOUTA, va)  # set vouta to 552mV
print(f"请测量任一tile的电压源，检查VOUTA的值是否为 {va} mV，如果正确，请点选对应的CheckBox")

a111sdk.a111_vsource_set(a111sdk.VOUTB, vb)  # set voutb to 958mV
print(f"请测量任一tile的电压源，检查VOUTB的值是否为 {vb} mV，如果正确，请点选对应的CheckBox")

a111sdk.a111_vsource_set(a111sdk.VOUTC, vc)  # set voutc to 200mV
print(f"请测量任一tile的电压源，检查VOUTC的值是否为 {vc} mV，如果正确，请点选对应的CheckBox")

a111sdk.a111_vsource_set(a111sdk.VOUTD, vd)  # set voutd to 1000mV
print(f"请测量任一tile的电压源，检查VOUTD的值是否为 {vd} mV，如果正确，请点选对应的CheckBox")

a111sdk.a111_vsource_set(a111sdk.VOUTE, ve)  # set voute to 5000mV
print(f"请测量任一tile的电压源，检查VOUTE的值是否为 {ve} mV，如果正确，请点选对应的CheckBox")

def checkvsource(**kwargs):
#     print([(k,v) for k, v in kwargs.items()])
    a = ([(k,v) for k, v in kwargs.items()])
    if a[0][1] and a[1][1] and a[2][1] and a[3][1] and a[4][1]:
        print("测试结束，关闭设备")
        a111sdk.close_a111()

widgets.interact(checkvsource, **{c.description: c.value for c in chk})

### 电压源获取（电压单位：mV）

关于电压源的获取，有两种方式：
- 通过python接口函数
- 通过linux终端命令

下面先简单介绍一下命令行方式获取：
<div><br /><a href="pic/vsrc3.png"><img style="float: left;" src="pic/vsrc3.png" width=700 title="点击放大看原图"></a><br /></div>

接下来是python api方式获取电压源的值

In [ ]:
import time
import ipywidgets as widgets
import a111sdk

va = 552 
vb = 958 
vc = 200 
vd = 1000
ve = 5000

# 打开设备（根据驱动的设计，设备打开后会自动上电，因此不需要主动调用“a111_power_on()”）
a111sdk.open_a111()

a111sdk.a111_vsource_set(a111sdk.VOUTA, va)  # set vouta to 552mV
a111sdk.a111_vsource_set(a111sdk.VOUTB, vb)  # set voutb to 958mV
a111sdk.a111_vsource_set(a111sdk.VOUTC, vc)  # set voutc to 200mV
a111sdk.a111_vsource_set(a111sdk.VOUTD, vd)  # set voutd to 1000mV
a111sdk.a111_vsource_set(a111sdk.VOUTE, ve)  # set voute to 5000mV

print("等待2秒。。。")
time.sleep(2)
print("开始读取：")

a = a111sdk.a111_vsource_get(a111sdk.VOUTA)  # 
b = a111sdk.a111_vsource_get(a111sdk.VOUTB)  #
c = a111sdk.a111_vsource_get(a111sdk.VOUTC)  # 
d = a111sdk.a111_vsource_get(a111sdk.VOUTD)  # 
e = a111sdk.a111_vsource_get(a111sdk.VOUTE)  # 

bias_a = abs(a-va)
bias_b = abs(b-vb)
bias_c = abs(c-vc)
bias_d = abs(d-vd)
bias_e = abs(e-ve)

if bias_a <= 2 and bias_b <= 2 and bias_c <= 2 and bias_d <= 2 and bias_e <= 2:
    print(f"测试结束，所有值都正确, bias = {bias_a}/{bias_b}/{bias_c}/{bias_d}/{bias_e}")
else:
    print(f"Warning：电压值校验失败，请检查！ bias = {bias_a}/{bias_b}/{bias_c}/{bias_d}/{bias_e}")

a111sdk.close_a111()

## 边界测试与压力测试

### AD5686R

***根据ad5686r的[芯片手册](https://www.analog.com/media/en/technical-documentation/data-sheets/AD5686R_5685R_5684R.pdf)，它一共可以提供4路电压输出，对应我们程序中 a111sdk.VOUTA ~ a111sdk.VOUTD***
<div><a href="pic/vsrc4.png"><img style="float: left;border:3px solid blue" src="pic/vsrc4.png" width=1000 title="点击放大看原图"></a><br /></div>


根据GAIN pin的状态，我们可以得到两种最大输出电压：

- GAIN pin接 $GND$，最大输出电压为2.5V；
- GAIN pin接 $V_{LOGIC}$，最大输出电压为2.5V；

**根据目前的硬件设计，ad5686r的最大额定输出电压为2.5V；**

<div><a href="pic/vsrc5.png"><img style="float: left;border:2px solid blue" src="pic/vsrc5.png" width=400 title="点击放大看原图"></a><br /></div>

$$
\newline
$$



### AD5761R

ad5761r（[用户手册](https://www.analog.com/media/en/technical-documentation/data-sheets/ad5761r_5721r.pdf)）用于控制我们程序中的电压源  $a111sdk.VOUTE$；

<div><a href="pic/vsrc6.png"><img style="float: left;border:2px solid blue" src="pic/vsrc6.png" width=400 title="点击放大看原图"></a><br /></div>

与ad5686r不同，其输出电压的范围可以通过命令方式动态控制：

<div>
<div><a href="pic/vsrc7.png"><img style="float: left;" src="pic/vsrc7.png" width=900 title="点击放大看原图"></a><br /></div>

<div><a href="pic/vsrc8.png"><img style="float: left;" src="pic/vsrc8.png" width=800 title="点击放大看原图"></a><br /></div>
</div>



***根据产品的实际需求，我们将ad5761r的输出范围设置为-10V到+10V，这个设置是在驱动软件内部完成的，属于默认设置，未对用户侧开放；***

***

*下面代码可以为不同电压源设置不同的电压值*

In [6]:
import ipywidgets as widgets
import a111sdk

# 打开设备（根据驱动的设计，设备打开后会自动上电，因此不需要主动调用“a111_power_on()”）
a111sdk.open_a111()

def test_vsrc(x, y):
    vsrc=a111sdk.VOUTA
    vname="VOUTA"
    if x==1:
        vsrc=a111sdk.VOUTA
        vname="VOUTA"
    elif x==2:
        vsrc=a111sdk.VOUTB
        vname="VOUTB"
    elif x==3:
        vsrc=a111sdk.VOUTC
        vname="VOUTC"
    elif x==4:
        vsrc=a111sdk.VOUTD
        vname="VOUTD"
    elif x==5:
        vsrc=a111sdk.VOUTE
        vname="VOUTE"
    elif x==6:
        a111sdk.close_a111()
        return
    
    if y != 0:
        a111sdk.a111_vsource_set(vsrc, y)
        print(f"电压源{vname}已被设置为{y} mV，请测量电路板上的实际电压值！")
    return

widgets.interact(test_vsrc, x={"选择电压源" : 0, "VOUTA" : 1, "VOUTB" : 2, "VOUTC" : 3, "VOUTD" : 4, "VOUTE" : 5, "测试结束":6}, 
                            y={"选择电压值":0, "150mV":150, "250mV":250, "500mV":500, "800mV":800, 
                                               "1500mV":1500, "2500mv":2500, "5000mV":5000, "8000mV":8000,
                                               "12000mV":12000})


设备已打开！


interactive(children=(Dropdown(description='x', options={'选择电压源': 0, 'VOUTA': 1, 'VOUTB': 2, 'VOUTC': 3, 'VOUT…

<function __main__.test_vsrc(x, y)>